In [1]:
import pandas as pd

We have three databases: drugs.com reviews, FDA adverse events (FAERS), and National Average Drug Acquisition Costs (NADAC). We would like to be able to consistently pull records from all three databases, but they report drug names differently, and sometimes even within a single database the drug names are listed differently. We would like to create a "clean" column that links each database based on a simplified drug name. Since brand names for a drug can differ, we will focus on the active ingredient for a drug.

In [62]:
faers = pd.read_csv('../data/raw/DRUG18Q4.txt', sep='$')

In [ ]:
nadac = pd.read_csv('../data/raw/NADAC__National_Average_Drug_Acquisition_Cost_.csv')

In [ ]:
drugscom = df = pd.read_csv('../data/raw/drugsComTest_raw.tsv', sep="\t")

Since the FAERS database has a curated list of active ingredients, we can use it to generate an intial list of potential keys. There are many active ingredients fields containing hard to parse information, as well as just being NaN, so we can cut those out.

In [143]:
faers.head()

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
0,100035916,10003591,1,PS,GILENYA,FINGOLIMOD HYDROCHLORIDE,1,Oral,QD,NaN,NaN,NaN,NaN,NaN,NaN,22527.0,NaN,NaN,CAPSULE,QD
1,100050413,10005041,1,PS,PLAN B ONE-STEP,LEVONORGESTREL,1,Oral,1.5 MILLIGRAM DAILY;,NaN,NaN,D,NaN,NaN,NaN,21998.0,1.5,MG,TABLET,QD
2,1000551312,10005513,1,PS,ENBREL,ETANERCEPT,1,Subcutaneous,"50 MG, ONCE WEEKLY",50.0,MG,U,NaN,G79072,NaN,103795.0,50.0,MG,SOLUTION FOR INJECTION IN PRE-FILLED SYRINGE,/wk
3,1000551312,10005513,2,SS,ENBREL,ETANERCEPT,1,Unknown,"50 MG, ONCE WEEKLY (EVERY THURSDAY)",50.0,MG,U,NaN,S77448,NaN,103795.0,50.0,MG,SOLUTION FOR INJECTION IN PRE-FILLED SYRINGE,/wk
4,1000551312,10005513,3,SS,ENBREL,ETANERCEPT,1,Unknown,"1 DF, WEEKLY",50.0,MG,U,NaN,NaN,NaN,103795.0,1.0,DF,SOLUTION FOR INJECTION IN PRE-FILLED SYRINGE,/wk


In [142]:
print(faers[faers.prod_ai.str.contains(r'[-]', na = False)].prod_ai[1:3])
print(faers[faers.prod_ai.str.contains(r'[\\]', na = False)].prod_ai[1:3])
print(faers[faers.prod_ai.str.contains(r'[\.]', na = False)].prod_ai[1:3])
print(faers[faers.prod_ai.str.contains(r'[/]', na = False)].prod_ai[1:3])
print(faers[faers.prod_ai.str.contains(r'[(]', na = False)].prod_ai[1:3])

308    INFLUENZA A VIRUS A/CALIFORNIA/7/2009(H1N1)-LI...
309    INFLUENZA A VIRUS A/CALIFORNIA/7/2009(H1N1)-LI...
Name: prod_ai, dtype: object
25           SULFAMETHOXAZOLE\TRIMETHOPRIM
26    ACETAMINOPHEN\HYDROCODONE BITARTRATE
Name: prod_ai, dtype: object
414     TOCOPHERYL NICOTINATE, D-.ALPHA.
2897                  .ALPHA.-TOCOPHEROL
Name: prod_ai, dtype: object
309     INFLUENZA A VIRUS A/CALIFORNIA/7/2009(H1N1)-LI...
4017    VARICELLA-ZOSTER VIRUS STRAIN OKA/MERCK LIVE A...
Name: prod_ai, dtype: object
309     INFLUENZA A VIRUS A/CALIFORNIA/7/2009(H1N1)-LI...
1324    HEPATITIS A VIRUS STRAIN HM175 ANTIGEN (FORMAL...
Name: prod_ai, dtype: object


In [64]:
faers_cut = faers[pd.notnull(faers.prod_ai)]
faers_cut = faers_cut[~faers_cut.prod_ai.str.contains(r'[-\\\./(]', na = False)]
print((len(faers) - len(faers_cut))/len(faers))
print((len(faers.prod_ai.unique()) - len(faers_cut.prod_ai.unique()))/len(faers.prod_ai.unique()))

0.10224619949768399
0.31419678367629256


This removed 10% of entries from the database, but led to a reduction of 31% in active ingredients, indicating many were low frequency entries, supporting the idea that they were malformed and likely not useful for later analysis.

In [77]:
active_ingredients = [x.upper() for x in sorted(faers_cut.prod_ai.unique())]

In [78]:
len(active_ingredients)

3966

This gives us a list of 4000 active ingredients that covers 90% of the original database. Now we can try to use this list to link to the other databases.

In [71]:
nadac.head()

,NDC_Description,NDC,NADAC_Per_Unit,Effective_Date,Pricing_Unit,Pharmacy_Type_Indicator,OTC,Explanation_Code,Classification_for_Rate_Setting,Corresponding_Generic_Drug_NADAC_Per_Unit,Corresponding_Generic_Drug_Effective_Date,As of Date
0,IBUPROFEN 200 MG TABLET,70000017504,0.02991,11/21/2018,EA,C/I,Y,1,G,NaN,NaN,12/12/2018
1,OXYCODONE HCL ER 80 MG TABLET,115156201,9.02989,11/21/2018,EA,C/I,N,"1, 6",G,NaN,NaN,12/12/2018
2,OXYCODONE HCL 15 MG TABLET,603499128,0.15188,11/21/2018,EA,C/I,N,1,G,NaN,NaN,12/12/2018
3,KETOCONAZOLE 200 MG TABLET,51672402601,1.68457,11/21/2018,EA,C/I,N,1,G,NaN,NaN,12/12/2018
4,NICOTINE 7 MG/24HR PATCH,70000011302,1.72464,11/21/2018,EA,C/I,Y,"1, 6",G,NaN,NaN,12/12/2018


Since the NADAC database includes dosage and delivery method in the drug description column, we need to make a new column that tries to pull out just the drug name itself. We can start with a naive method that just takes the first word of the description column, although this will necessarily miss any drug name that consists of more than one word.

In [72]:
nadac['drugName'] = nadac.apply(lambda row: row.NDC_Description.split()[0], axis = 1)

In [84]:
print(nadac[nadac.drugName.str.contains(r'[-]', na = False)].drugName[1:3])
print(nadac[nadac.drugName.str.contains(r'[\.]', na = False)].drugName[1:3])
print(nadac[nadac.drugName.str.contains(r'[(]', na = False)].drugName[1:3])

33         THEO-24
47    LORATADINE-D
Name: drugName, dtype: object
8822    E.E.S.
8950       DR.
Name: drugName, dtype: object
67962    HUMIRA(CF)
76717    HUMIRA(CF)
Name: drugName, dtype: object
Series([], Name: drugName, dtype: object)


In [85]:
nadac_cut = nadac[pd.notnull(nadac.drugName)]
nadac_cut = nadac_cut[~nadac_cut.drugName.str.contains(r'[-\\\./(]', na = False)]
print((len(nadac) - len(nadac_cut))/len(nadac))
print((len(nadac.drugName.unique()) - len(nadac_cut.drugName.unique()))/len(nadac.drugName.unique()))

0.09111165336154577
0.18292682926829268


This removed ~9% of entries from the nadac database, but about 18% of unique drugs. Now we need to check the overlap with our active ingredients list.

In [114]:
1 - (len(nadac_cut) - len(nadac_cut[nadac_cut.drugName.isin(active_ingredients)]))/len(nadac_cut)

0.6391725030705994

In [97]:
nadac_cut[~nadac_cut.drugName.isin(active_ingredients)].drugName[1:10]

15    AMLODIPINE
16        LIVALO
19       ENSKYCE
26            HM
29    NICORELIEF
32            GS
39      CYCLOSET
42         MAPAP
46          TUMS
Name: drugName, dtype: object

Unfortunately, only 63% of the drugName entries in the NADAC database match an entry in our active ingredients list. If we look at the entries that don't match, we see many brand names, which may have an active ingredient that match. To get these entries as well, we will need to map brand names to active ingredients.

In [118]:
brand_names = {x.drugname:x.prod_ai for x in faers_cut[faers_cut.prod_ai.isin(active_ingredients)].itertuples()}
drug_names = brand_names
for ai in active_ingredients:
    drug_names[ai] = ai

In [119]:
1 - (len(nadac_cut) - len(nadac_cut[(nadac_cut.drugName.isin(drug_names))]))/len(nadac_cut)

0.8602464395432481

By including brand names, we now have 86% coverage in the NADAC database. We can now create a new column containing the parsed active ingredient.

In [121]:
nadac_cut['prod_ai'] = nadac_cut.apply(lambda row: drug_names[row.drugName] if row.drugName in drug_names else 'NA', axis = 1)

In [139]:
acetaminophen_faers = faers_cut[faers_cut.prod_ai == 'ACETAMINOPHEN']
acetaminophen_nadac = nadac_cut[nadac_cut.prod_ai == 'ACETAMINOPHEN']

In [140]:
acetaminophen_faers[1:5]

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
133,100190523,10019052,5,SS,ACETAMINOPHEN.,ACETAMINOPHEN,1,Unknown,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,30.0,G,UNSPECIFIED,NaN
168,100268942,10026894,13,C,PARACETAMOL,ACETAMINOPHEN,1,Unknown,NaN,NaN,NaN,D,NaN,NaN,NaN,NaN,1.0,G,TABLET,TID
439,1004975117,10049751,43,C,PYRINAZIN,ACETAMINOPHEN,1,Oral,"0.5 MG, ONCE DAILY",0.5,MG,D,NaN,NaN,NaN,NaN,0.5,MG,NaN,QD
440,1004975117,10049751,44,C,PYRINAZIN,ACETAMINOPHEN,1,Oral,"0.5 MG, ONCE DAILY",0.5,MG,D,NaN,NaN,NaN,NaN,0.5,MG,NaN,QD


In [141]:
acetaminophen_nadac[1:5]

,NDC_Description,NDC,NADAC_Per_Unit,Effective_Date,Pricing_Unit,Pharmacy_Type_Indicator,OTC,Explanation_Code,Classification_for_Rate_Setting,Corresponding_Generic_Drug_NADAC_Per_Unit,Corresponding_Generic_Drug_Effective_Date,As of Date,drugName,prod_ai
470,ACEPHEN 650 MG SUPPOSITORY,713016512,0.32045,11/21/2018,EA,C/I,Y,1,G,NaN,NaN,12/12/2018,ACEPHEN,ACETAMINOPHEN
627,ACETAMINOPHEN 500 MG TABLET,904673080,0.02684,11/21/2018,EA,C/I,Y,1,G,NaN,NaN,12/12/2018,ACETAMINOPHEN,ACETAMINOPHEN
1176,ACETAMINOPHEN 500 MG GELCAP,70000031202,0.02684,12/05/2018,EA,C/I,Y,1,G,NaN,NaN,12/12/2018,ACETAMINOPHEN,ACETAMINOPHEN
1365,ACETAMINOPHEN 500 MG CAPLET,70000015102,0.02684,11/21/2018,EA,C/I,Y,1,G,NaN,NaN,12/12/2018,ACETAMINOPHEN,ACETAMINOPHEN
